In [1]:
import numpy as np
import pandas as pd
from simple.renko import Renko
from simple.plotly import interactFigure

In [2]:
T = np.load('data/ETHUSDT.2022-05-08.tick.npz')['ETHUSDT'].view(np.recarray)
len(T)

907307

In [3]:
def model(step: int = (1, 20, 1)):
    rnk = Renko(step, T['Price'])
    rnk.create_renko()

    X = pd.DataFrame(rnk.bricks)
    X['mid'] = (X.close + X.open) / 2
    renko = X.mid.values

    return 0, locals()

In [6]:
interactFigure(model, renko={'color': 'gray', 'opacity': 0.4, 'mode': 'markers+lines'})